In [ ]:
import numpy as np
from easyvec import Vec3, Mat3
from interpolation import Interp1d, Interp2d

In [ ]:
# _validate_y
#  get_angle те ли формулы?
# Силы и моменты
# axis
# Коэфты Сх, Су_alpha, Cy_delta...
# self.delta_y, self.delta_z

In [ ]:
opts = {
    'd': ,
    'L': ,
    'm_0': ,
    'v_0': ,
    't_marsh': ,
    'w_marsh': ,
    'P_marsh': ,
    'record': False
}

In [ ]:
class Missile3D(object):
    
    @classmethod
    def get_missile(cls, opts):
        """
        Классовый метод создания ракеты со всеми необходимыми аэродинамическими, массо- и тяговременными характеристиками
        
        Arguments
        ---------
        opts : dict 
               словарь с параметрами ракеты
               
        Returns
        -------
        missile : cls
                  экземпляр класса Missile3D
        """
        @np.vectorize
        def get_m(t):
            if t < t_marsh:
                return m_0 - G_marsh * t
            else:
                return m_0 - w_marsh
            
        @np.vectorize
        def get_P(t):
            if t < t_marsh:
                return P_marsh
            else:
                return 0
        
        @np.vectorize
        def get_J(t):
            if t < t_marsh:
                Jx = 1/2 * (m_0 - G_marsh * t) * (d/2)**2
                Jy = 1/12 * (m_0 - G_marsh * t) * (L)**2
                Jz = 1/12 * (m_0 - G_marsh * t) * (L)**2
            else:
                Jx = 1/2 * (m_0 - w_marsh) * (d/2)**2
                Jy = 1/12 * (m_0 - w_marsh) * (L)**2
                Jz = 1/12 * (m_0 - w_marsh) * (L)**2
            return np.array([Jx, Jy, Jz])
        
        d       = opts['d']
        L       = opts['L']
        m_0     = opts['m_0']
        v_0     = opts['v_0']
        v_mean  = opts['v_mean']
        t_marsh = opts['t_marsh']
        w_marsh = opts['w_marsh']
        P_marsh = opts['P_marsh']
        G_marsh = w_marsh / t_marsh
        vel_wind = opts.get('vel_wind', [0,0,0])
        postProcessing = opts.get('record', True)
        
        # TODO временно константы грубо говря подставить arr_alpha, arr_mach, arr_cx, arr_cya
        
        ts    = np.linspace(0, t_marsh, 100)
        m_itr = Interp1d(ts, get_m(ts))
        J_itr = Interp1d(ts, get_J(ts))
        P_itr = Interp1d(ts, get_P(ts))
        atm_itr = table_atm
        Cx_itr = Interp2d(arr_alpha, arr_mach, arr_cx)
        Cy_alpha_itr = Interp1d(arr_mach, arr_cy_alpha)
        Cy_delta_itr = Interp1d(arr_mach, arr_cy_delta)
        
        missile = cls(d=d, v_0=v_0, v_mean=v_mean, P_itr=P_itr, m_itr=m_itr, J_itr=J_itr,
                      Cx_itr=Cx_itr, Cy_alpha_itr=Cy_alpha_itr, Cy_delta_itr=Cy_delta_itr, atm_itr=atm_itr, vel_wind=vel_wind,
                      postProcessing=postProcessing)
        
        return missile
    
    def __init__(self, qw=1, qx=0, qy=0, qz=0, g=9.80665, n=14, **kwargs):
        """
        Конструктор класса Missile3D
        
        Arguments
        ---------
        
        ...
        
        """
        self.g = g
        self.t = 0
        self.n = n
        self.state = np.zeros(self.n)
        self.state_init = np.zeros(self.n)
                
        self.d       = kwargs['d']
        self.S_mid   = np.pi * d**2 / 4
        self.v       = kwargs['v_0']
        self.v_0     = kwargs['v_0']
        self.v_mean  = kwargs['v_mean']
        self.thetta  = kwargs['thetta']
        self.psi     = kwargs['psi']
        self.gamma   = kwargs['gamma']
        self.P_itr   = kwargs['P_itr']
        self.m_itr   = kwargs['m_itr']
        self.Cx_itr  = kwargs['Cx_itr']
        self.Cy_alpha_itr = kwargs['Cy_alpha_itr']
        self.Cy_delta_itr = kwargs['Cy_delta_itr']
        self.atm_itr = kwargs['atm_itr']
        self.J_itr   = kwargs['J_itr']
        self.vel_wind = kwargs['vel_wind']
        
        self.postProcessing = kwargs['postProcessing']
        
        self.Fx = []
        self.Fy = []
        self.Fz = []
        self.Mx = []
        self.My = []
        self.Mz = []
        self.angle = []
        
        # Инициилизация кватерниона 
        if ("scalar" in kwargs) or ("vector" in kwargs):
            # с помощью скалярной и векторной части
            scalar = kwargs.get("scalar", 0.0)
            if scalar is None:
                scalar = 0.0
            else:
                scalar = float(scalar)
            vector = kwargs.get("vector", [])
            vector = self._validate_number_sequence(vector, 3)
            self.qw = scalar
            self.qx, self.qy, self.qz = vector
        elif ("thetta" in kwargs) or ("psi" in kwargs) or ("gamma" in kwargs):
            # с помощью самолетных углов
            thetta = np.radians(kwargs.get("thetta", 0.0))
            psi = np.radians(kwargs.get("psi", 0.0))
            gamma = np.radians(kwargs.get("gamma", 0.0))
            self.qw = np.cos(thetta/2) * np.cos(psi/2) * np.cos(gamma/2) + np.sin(thetta/2) * np.sin(psi/2) * np.sin(gamma/2)
            self.qx = np.cos(thetta/2) * np.cos(psi/2) * np.sin(gamma/2) - np.sin(thetta/2) * np.sin(psi/2) * np.cos(gamma/2)
            self.qy = np.cos(thetta/2) * np.sin(psi/2) * np.cos(gamma/2) + np.sin(thetta/2) * np.cos(psi/2) * np.sin(gamma/2)
            self.qz = np.sin(thetta/2) * np.cos(psi/2) * np.cos(gamma/2) - np.cos(thetta/2) * np.sin(psi/2) * np.sin(gamma/2)
        elif "array" in kwargs:
            self.qw, self.qx, self.qy, self.qz = self._validate_number_sequence(kwargs["array"], 4)
        else:
            # по умолчанию
            self.qw = qw
            self.qx = qx
            self.qy = qy
            self.qz = qz
        self._normalise_quaternion()
    
    @property
    def rho(self):
        return self.atm_itr(self.state[1], 3)
    
    @property
    def sonic_speed(self):
        return self.atm_itr(self.state[1], 4)
    
    @property
    def q_abs(self):
        """
        Текущий скоростной напор набегающего потока воздуха
        
        Returns
        -------
        q : float
        """
        return self.rho * self.vel_abs**2 / 2
    
    @property
    def mach_abs(self):
        """
        Текущее абсолютное значение числа Маха
        
        Returns
        -------
        M : float
        """
        return self.vel_abs / self.atm_itr(self.state[1], 4)
    
    @property
    def mach(self):
        """
        Текущее вектор числа Маха
        
        Returns
        -------
        M : np.ndarray
        [Mx, My, Mz]
        [0,   1,  2]
        """
        return np.array(self.vel / self.atm_itr(self.state[1], 4))
    
    @property
    def quaternion(self):
        """
        Кватернион результируюшего текущего поворота (кватернион как четырехмерный вектор)
        
        Returns
        -------
        quaternion : np.ndarray
                     q = [qw, qx, qy, qz] 
                         [ 0,  1,  2,  3]
        """
        return np.array([self.qw, self.qx, self.qy, self.qz])
    
    @property
    def norm_quaternion(self):
        """
        Свойство, возвращающее норму кватерниона (длину)
        """
        return np.sqrt(np.dot(self.quaternion, self.quaternion))

    @property
    def thetta(self):
        """
        Угол тангажа ракеты
        """
        return self.get_angle()[0]
    
    @property
    def psi(self):
        """
        Угол курса ракеты
        """
        return self.get_angle()[1]
    
    @property
    def gamma(self):
        """
        Угол крена ракеты
        """
        return self.get_angle()[2]
    
    @property
    def alpha(self):
        """
        Угол атаки корпуса ракеты в пространстве
        """
        return self.get_alpha()[0]
    
    @property
    def alpha_k(self):
        """
        Угол атаки корпуса ракеты
        """
        return self.get_alpha()[0][0]
    
    @property
    def alpha_y(self):
        """
        Угол атаки корпуса ракеты в вертикальной плоскости
        """
        return self.get_alpha()[0][1]
    
    @property
    def alpha_z(self):
        """
        Угол атаки корпуса ракеты в горизонтальной плоскости
        """
        return self.get_alpha()[0][2]
    
    @property
    def gamma_alpha(self):
        """
        Угол ориентации атаки корпуса ракеты
        """
        return self.get_alpha()[1]
    
    @property
    def pos(self):
        """
        Пространственный вектор положения ракеты в ГСК
        
        Returns
        -------
        cls Vec3
        [x, y, z] 
        [0, 1, 2]
        """
        return Vec3(self.state[0], self.state[1], self.state[2])
    
    @property
    def axis(self):
        """
        Вектор оси ракеты - проекция в ИСК
        """
        pass # TODO ось ракеты вернуть
    
    @property
    def vel(self):
        """
        Пространственный вектор скорости ракеты
        
        Returns
        -------
        cls Vec3
        [vx, vy, vz] 
        [0,   1,  2]
        """
        return Vec3(self.state[3], self.state[4], self.state[5])
    
    @property
    def vel_abs(self):
        return np.sqrt(self.vel.dot(self.vel))
    
    @property
    def vel_stream(self):
        return self.get_vel_stream()
    
    @property
    def w(self):
        """
        Пространственный вектор углового вращения ракеты в ССК
        
        Returns
        -------
        cls Vec3
        [wx, wy, wz] 
        [0,   1,  2]
        """
        return Vec3(self.state[6], self.state[7], self.state[8])
    
    def f_system(self, t, y):
        """
        Функция, записывающая правую часть системы ОДУ динамики материального тела в пространстве.
        
        Arguments
        ---------
        t : float
            Время в секундах
        y : np.ndarray / list
            Вектор состояния системы 
            [0, 1, 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]
            [x, y, z, vx, vy, vz, wx, wy, wz, qw, qx, qy, qz,  t]     
            
        Returns
        -------
        dy : np.ndarray / list
             Вектор состояния после дифференцирования системы dy/dt
             [0,   1,  2,  3,  4,  5,    6,    7,    8,   9,  10,  11,  12, 13]
             [vx, vy, vz, ax, ay, az, epsx, epsy, epsz, dqw, dqx, dqy, dqz,  1]
        """
        x, y, z, vx, vy, vz, qw, qx, qy, qz, wx, wy, w3, t = y
                
        J_x, J_y, J_z = self.J_itr(t)
                
        vel_stream_x, vel_stream_y, vel_stream_z = self.get_vel_stream()
        
        if self.vel_abs < 1e-5:
            Xx, Xy, Xz = 0, 0, 0
        else:
            mach_x = vel_stream_x / self.sonic_speed
            mach_y = vel_stream_y / self.sonic_speed
            mach_z = vel_stream_z / self.sonic_speed
            Xx = self.Cx_itr(mach_x, self.alpha_y) * self.rho * vel_stream_x**2 / 2 * self.S_mid # Мда от чего Сх хуй пойми
            Xy = # TODO
            Xz = # TODO

        # Нормальная АД сила    
        Y = self.Cy_alpha_itr(self.mach_abs) * self.alpha_y * self.q * self.S_mid * 57.3 + \ # точно ли от абс значения а не от проекции
             self.Cy_delta_itr(self.mach_abs) * self.delta_y * self.q * self.S_mid * 57.3
        Z = self.Cy_alpha_itr(self.mach_abs) * self.alpha_z * self.q * self.S_mid * 57.3 + \
             self.Cy_delta_itr(self.mach_abs) * self.delta_z * self.q * self.S_mid * 57.3
        
        # Сумма проекций
        Fx = self.P_itr(t) + Xx
        Fy = Y + Xy
        Fz = Z + Xz
        
        # из ССК в ИСК
        F_sum = self._matrix_transition() * Vec3(Fx, Fy, Fz)
        Fx, Fy, Fz = F_sum
        
        Mx = # TODO
        My = # TODO
        Mz = # TODO
        
        dy = np.zeros(self.n)

        # dr/dt
        dy[0] = vx
        dy[1] = vy
        dy[2] = vz
        
        # dv/dt = a
        dy[3] = Fx / self.m_itr(t)
        dy[4] = Fy / self.m_itr(t) - self.g
        dy[5] = Fz / self.m_itr(t)
        
        # domega/dt = eps
        dy[6] = (Mx + (J_y - J_z) * wy * wz) / J_x 
        dy[7] = (My + (J_z - J_x) * wz * wx) / J_y
        dy[8] = (Mz + (J_x - J_y) * wx * wy) / J_z
        
        # dq/dt
        dy[9] = -1/2 * (qx * wx + qy * wy + qz * wz) 
        dy[10] = 1/2 * (qw * wx - qz * wy + qy * wz)
        dy[11] = 1/2 * (qw * wy - qx * wz + qz * wx)
        dy[12] = 1/2 * (qw * wz - qy * wx + qx * wy)
        
        # dt/dt
        dy[13] = 1
        
        if self.postProcessing:
            self.Fx.append([t, Fx])
            self.Fy.append([t, Fy])
            self.Fz.append([t, Fz])
            self.Mx.append([t, Mx])
            self.My.append([t, My])
            self.Mz.append([t, Mz])
            self.angle.append([t, self.get_angle()])
        
        return dy
       
    def step(self, tau=0.1, n=10):
        """
        Метод моделирования динамики ракеты за шаг по времени tau
        На протяжении tau управляющее воздействие на ракету постоянно (action)
        Шаг интегирования dt определяется из числа разбиения n временного шага tau
        Функция меняет внутреннее состояние ракеты state на момент окончания шага
        
        Arguments
        ---------
        action : int
                 управляющее воздействие на протяжении шага
        tau    : float 
                 длина шага по времени (не путать с шагом интегрирования)
        n      : int
                 число шагов разбиений интервала времени tau
            
        Returns
        -------
        None
        """
        t = self.state[-1]  
        dt = tau / n
        for i in range(n):
            k1 = self.f_system(t, self.state)
            k2 = self.f_system(t + 0.5 * dt, self._validate_y(self.state + 0.5 * dt * k1))
            k3 = self.f_system(t + 0.5 * dt, self._validate_y(self.state + 0.5 * dt * k2))
            k4 = self.f_system(t + dt,       self._validate_y(self.state + dt * k3))
            y = self._validate_y(self.state + dt / 6 * (k1 + 2 * k2 + 2 * k3 + k4))
            t += dt
            self.t = t
            self.state = y
            self.state[-1] = t
        
    def get_angle(self):
        """
        Метод, вычисляющий самолетные углы Крылова по кватерниону результирующего поворота
        
        Returns
        -------
        np.ndarray
        [thetta, psi, gamma]
        """
        qw, qx, qy, qz = self.quaternion
        thetta = np.arctan(2*(qx*qy+qw*qz)/(2*(qw**2+qx**2)-1)) # np.arcsin(2*(qx*qz-qw*qy))
        psi = np.arcsin(2*(qw*qy-qx*qz)) # - np.arctan(2*(qx*qz-qw*qy)/(2*(qw**2+qx**2)-1))
        gamma = np.arctan(2*(qy*qz+qw*qx)/(2*(qw**2+qz**2)-1)) # - np.arctan(2*(qy*qz-qw*qx)/(2*(qw**2+qy**2)-1))
        return np.array([thetta, psi, gamma])
    
    def get_alpha(self):
        """
        Метод, вычисляющий угол атаки корпуса ракеты (проекция на ССК) и его угол ориентации
        Угол атаки - угол между продольной осью ракеты (OX) и вектором скорости набегающего потока
        Угол ориентации угла атаки - угол, определяющий ориентацию относительно плоскостей крыльев (рулей)
        
        Arguments
        ---------
        None
        
        Returns
        -------
        (                 np.ndarray,       float)
        ([alpha_k, alpha_y, alpha_z], gamma_alphs)
        """
        vel_stream_x, vel_stream_y, vel_stream_z = self.get_vel_stream()
        vel_stream = np.linalg.norm(self.get_vel_stream())
        if vp_x >= 0:
            alpha_k = np.arcsin((vel_stream_y**2 + vel_stream_z**2) / vel_stream)
            alpha_y = -np.arcsin(vel_stream_y / np.sqrt(self.vel.x**2 + self.vel.y**2))
            alpha_z = -np.arcsin(vel_stream_z / np.sqrt(self.vel.x**2 + self.vel.z**2))
        else:
            alpha_k = np.pi - np.arcsin((vel_stream_y**2 + vel_stream_z**2) / vel_stream)
            alpha_y = -np.pi * np.sign(vel_stream_y) + np.arcsin(vel_stream_y / np.sqrt(self.vel.x**2 + self.vel.y**2))
            alpha_z = -np.pi * np.sign(vel_stream_z) + np.arcsin(vel_stream_z / np.sqrt(self.vel.x**2 + self.vel.z**2))
        gamma_alphs = np.arctg(abs(vel_stream_z / vel_stream_y))
        return np.array([alpha_k, alpha_y, alpha_z]), gamma_alphs
    
    def get_vel_stream(self):
        """
        Метод, возвращающий скорость набегающего потока в ССК
        
        Arguments
        ---------
        None
                   
        Returns
        -------
        Vec3 
        [vel_stream_x, vel_stream_y, vel_stream_z]
        """
        return С.T * (self.vel + self.vel_wind)
    
    def get_initial_parameters_of_missile(self, target):
        """
        Метод, возвращающий начальное состояние ракеты, направленной в упрежденную точку
        
        Arguments:
        ---------
        target : object cls target
        
        Returns
        -------
        np.ndarray [x, y, z, vx, vy, vz, wx, wy, wz, qw, qx, qy, qz, t]
        """
        meeting_point = self.get_instant_meeting_point(target)
        # доворачиваем ИСК так чтобы точка встречи была в вертикальной плоскости X0Y
        x_new = meeting_point[2] * np.sin(psi) + meeting_point[0] * np.cos(psi)
        y_new = y
        z_new = meeting_point[2] * np.cos(psi) - meeting_point[0] * np.sin(psi)
        meeting_point = np.array([x_new, y_new, z_new])
        visir_prevention = meeting_point - self.pos
        thetta = np.arctan2(visir_prevention[1], visir_prevention[0])
        qw, qx, qy, qz = self.get_init_quaternion(meeting_point)
        return np.array([0, 0, 0, self.v0 * np.cos(thetta), self.v0 * np.sin(thetta), 0, 0, 0, 0, qw, qx, qy, qz, 0])   
        
    def set_initial_parameters_of_missile(self, target, parameters_of_missile=None):
        """
        Метод, устанавливающий начальные параметры (положение, линейную скорость, угловую скорость, кватернион, время)
        
        Arguments:
        ---------
        target                : object cls target
        parameters_of_missile : list / np.ndarray (default : None)
                                [x, y, z, vx, vy, vz, wx, wy, wz, qw, qx, qy, qz, t]
        
        Returns
        -------
        None
        """
        if parameters_of_missile is None:
            parameters_of_missile = self.get_initial_parameters_of_missile(target)
        self.state = np.array(parameters_of_missile)
        self.state_init = np.array(parameters_of_missile)
    
    def get_line_visir(self, target_pos):
        """
        Метод, вычисляющий линию визирования ракета-цель
        
        Arguments:
        ---------
        target_pos : list/tuple/np.ndarray
                     координата цели
        
        Returns
        -------
        np.ndarray
        [x, y, z]
        """
        return np.array(target_pos - self.pos)
    
    def get_anlge_visir(self, target_pos):
        """
        Метод, вычисляющий углы линии визирования ракета-цель в вертикальной плоскости
        
        Arguments:
        ---------
        target_pos : list/tuple/np.ndarray
                     координата цели
        
        Returns
        -------
        float
        arctg(y/x)
        """
        visir = self.get_line_visir(target_pos)
        return np.arctan2(visir[1], visir[0])
    
    def get_angle_init(self, target_pos):
        """
        Метод, возвращающий значения углов положения ССК относительно ИСК в начальный момент времени
        
        Arguments:
        ---------
        target_pos : list/tuple/np.ndarray
                     координата цели
        
        Returns
        -------
        np.ndarray
        [thetta, psi, gamma] в радианах
        """
        return np.array([self.get_anlge_visir(target_pos), 0.0, 0.0])
    
    def get_init_quaternion(self, target_pos):
        """
        Метод, возвращающий значение начального кватерниона поворота ССК в ИСК
        
        Arguments:
        ---------
        target_pos : list/tuple/np.ndarray
                     координата цели
        
        Returns
        -------
        np.ndarray
        [qw, qx, qy, qz]
        """
        thetta, psi, gamma = self.get_angle_init(target_pos)
        qw = np.cos(thetta/2) * np.cos(psi/2) * np.cos(gamma/2) + np.sin(thetta/2) * np.sin(psi/2) * np.sin(gamma/2)
        qx = np.cos(thetta/2) * np.cos(psi/2) * np.sin(gamma/2) - np.sin(thetta/2) * np.sin(psi/2) * np.cos(gamma/2)
        qy = np.cos(thetta/2) * np.sin(psi/2) * np.cos(gamma/2) + np.sin(thetta/2) * np.cos(psi/2) * np.sin(gamma/2)
        qz = np.sin(thetta/2) * np.cos(psi/2) * np.cos(gamma/2) - np.cos(thetta/2) * np.sin(psi/2) * np.sin(gamma/2)
        norm_quaternion = np.sqrt(qw**2 + qx**2 + qy**2 + qz**2)
        qw /= norm_quaternion
        qx /= norm_quaternion
        qy /= norm_quaternion
        qz /= norm_quaternion
        return np.array([qw, qx, qy, qz])
    
    def set_init_quaternion(self, target_pos):
        """
        Метод, устанавливающий значение начального кватерниона поворота ССК в ИСК
        
        Arguments:
        ---------
        target_pos : list/tuple/np.ndarray
                     координата цели
        
        Returns
        -------
        None
        """
        qw, qx, qy, qz = self.get_init_quaternion(target_pos)
        self.qw, self.qx, self.qy, self.qz = qw, qx, qy, qz 
        self._normalise_quaternion()
        
    def get_instant_meeting_point(self, target):
        """
        Метод, находящий мгновенную точку встречи ракеты с целью
        
        Arguments
        ---------
        target : object cls target
                     
        Retuns
        ------
        np.ndarray
        координата точки встречи
        """
        trg_pos = np.array(target.pos)
        trg_vel = np.array(target.vel)

        vis  = self.get_line_visir(target)
        vis1 = vis / np.linalg.norm(vis)
        
        
        t = np.linalg.norm(vis) / np.linalg.norm(vel_close)
        return trg_pos + trg_vel * t

#         trg_vel_tau = np.dot(trg_vel, vis1) * vis1
#         trg_vel_n   = trg_vel - trg_vel_tau
#         trg_vel_n1  = trg_vel_n / np.linalg.norm(trg_vel_n)

#         if np.linalg.norm(trg_vel_n) > se:
#             return False, trg_pos

#         my_vel_n   = trg_vel_n
#         my_vel_tau = vis1 * sqrt(my_vel_abs**2 - np.linalg.norm(my_vel_n)**2)

#         vel_close = my_vel_tau - trg_vel_tau
#         if np.dot(vis1, vel_close) <= 0:
#             return trg_pos

#         t = np.linalg.norm(vis) / np.linalg.norm(vel_close)
        
#         return trg_pos + trg_vel * t
    
    def to_dict(self):
        return {
            't': self.t,
            'pos': self.pos,
            'vel': self.vel,
            'q': self.quaternion,
            'w': self.w
        }
    
    def to_numpy(self):
        return np.array(self.state)
       
    def _validate_number_sequence(self, seq, n):
        if seq is None:
            return np.zeros(n)
        if len(seq) == 0:
            return np.zeros(n)            
        elif len(seq) == n:
            try:
                l = [float(elem) for elem in seq]
            except ValueError:
                raise ValueError("Один или несколько элементов в <{!r}> не действительное(-ые) число(-а)".format(seq))
            else:
                return np.array(l)    
        else:
            raise ValueError("Неожиданное число элементов. Получено: {}, Ожидалось: {}.".format(len(seq), n))      
    
    def _normalise_quaternion(self, tolerance=1e-14):
        """
        Вспомогательный метод, нормализующий кватернион, если он таковым не является
        """
        if abs(1.0 - self.norm_quaternion) > tolerance:
            if self.norm_quaternion > 0:
                self.quaternion = self.quaternion / self.norm_quaternion
        
    def _matrix_transition(self):
        """
        Вспомогательный метод, образующий матрицу перехода от ССК к ИСК из параметров Родрига-Гамильтона
        
        Returns
        -------
        Mat3
        |C11, C12, C13|
        |C21, C22, C23|
        |C31, C32, C33|
        """
        C11 = 2 * (self.qw**2 + self.qx**2) - 1
        C12 = 2 * (self.qx * self.qy - self.qw * self.qz)
        C13 = 2 * (self.qx * self.qz + self.qw * self.qy)
        C21 = 2 * (self.qx * self.qy + self.qw * self.qz)
        C22 = 2 * (self.qw**2 + self.qy**2) - 1
        C23 = 2 * (self.qy * self.qz - self.qw * self.qx)
        C31 = 2 * (self.qx * self.qz - self.qw * self.qy)
        C32 = 2 * (self.qy * self.qz + self.qw * self.qx)
        C33 = 2 * (self.qw**2 + self.qz**2) - 1
        C1 = Vec3(C11, C12, C13)
        C2 = Vec3(C21, C22, C23)
        C3 = Vec3(C31, C32, C33)
        return Mat3(C1, C2, C3)
    
    def _validate_y(self, y):
        # TODO: проверка углов
        # return y
        pass